# Edit Notebooks for Further Analysis

Run this notebook to generate cleaned data for the Soc 130 module.

In [4]:
# !pip install --no-cache-dir -U -q folium
import pandas as pd
import geojson
import geopy
import numpy as np
import matplotlib.pyplot as plt
import folium
from datascience import *
from IPython.display import HTML, display, IFrame
from folium import plugins
%matplotlib inline
from sklearn import linear_model
from scipy import stats
import os
import ipywidgets as widgets
from soc_module import *
import random
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
sns.set()

from datetime import datetime

In [5]:
class_data = pd.read_csv('./data/fa18-soc-disorder-update.csv', index_col=0)
#class_data['Census Tract'] = class_data['Census Tract'].apply(fix_tract)  # fix census tract data type

class_data['Timestamp'] = class_data['Timestamp'].astype('str')
class_data = class_data.iloc[1:,:]

# rekey yes and no responses
for c in class_data.columns:
    if "Yes" in set(class_data[c]):
        class_data[c] = class_data[c].map({"Yes": 1, "No": 0})

# limit to submissions from Spring 2018
class_data['Timestamp'] = [datetime.strptime(time, '%m/%d/%Y %H:%M:%S') for time in class_data['Timestamp']]
class_data = class_data[[time.year == 2018 for time in class_data['Timestamp']]]
class_data = class_data.reset_index().drop('index', axis = 1)

image_data = class_data.iloc[:, 20:]
image_data = image_data.rename(columns = { 
    'Full Address of Block Face in Image #1 (Street Number, Street Name, City, State, Zip Code). E.g.: 2128 Oxford Street, Berkeley, CA, 94704.': 'Full Address of Photo #1 Location',
    'Full Address of Block Face in Image #2 (Street Number, Street Name, City, State, Zip Code). E.g.: 2128 Oxford Street, Berkeley, CA, 94704.': 'Full Address of Photo #2 Location',
    'Full Address of Block Face in Image #3 (Street Number, Street Name, City, State, Zip Code). E.g.: 2128 Oxford Street, Berkeley, CA, 94704.': 'Full Address of Photo #3 Location',
    'Full Address of Block Face in Image #4 (Street Number, Street Name, City, State, Zip Code). E.g.: 2128 Oxford Street, Berkeley, CA, 94704.': 'Full Address of Photo #4 Location',
    'Full Address of Block Face in Image #5 (Street Number, Street Name, City, State, Zip Code). E.g.: 2128 Oxford Street, Berkeley, CA, 94704.': 'Full Address of Photo #5 Location'
})

image_data['Census Tract'] = class_data['Census Tract']

In [6]:
image_data

,Other thoughts or comments for Image #1,Image #2,Full Address of Photo #2 Location,Other thoughts or comments for Image #2,Image #3,Full Address of Photo #3 Location,Other thoughts or comments for Image #3,Image #4,Other thoughts or comments for Image #4,Full Address of Photo #4 Location,Image #5,Full Address of Photo #5 Location,Other thoughts or comments for Image #5,Image #1 coordinates,Image #2 coordinates,Image #3 coordinates,Image #4 coordinates,Image #5 coordinates,Census Tract
0,I found it interesting because the man was ver...,https://drive.google.com/open?id=1Zd2LzopzPuDk...,"Touraine Hotel 1502-1583 Clay Street Oakland, CA",This building was intriguing because the archi...,https://drive.google.com/open?id=1u9fX5SCo4Vrx...,Cape & Cowl Restaurant. 1602 Clay Street Oakl...,This picture caught my attention because of th...,NaN,NaN,NaN,NaN,NaN,NaN,"[37.8431445, -122.283849]","(37.8087183, -122.2715003)","(37.8087183, -122.2715003)",NaN,NaN,4028.00
1,I found this graffiti on a wall in the parking...,https://drive.google.com/open?id=1-Kz4-Fbk5G7w...,"Channing Way and Shattuck Ave, Berkeley, CA, 9...",There is a Ford Bike Station and it is almost ...,https://drive.google.com/open?id=1zjA9f2t64BX0...,"2344 Fulton St, Berkeley, CA, 94704",This is a house for Cal's track and field team.,https://drive.google.com/open?id=1ZGfWt-nuNBqy...,"Someone put a sticker saying ""eating animals"" ...","Channing Way and Fulton St, Berkeley, CA, 94704",https://drive.google.com/open?id=1f2JyD7z6fSZC...,"2121 Channing Way, Berkeley, CA, 94704",There is a Buddhist Temple which shows the div...,"[37.8658887573699, -122.266275305354]","(37.8678899, -122.269247)","[37.8666288180719, -122.265876910725]","(37.8678899, -122.269247)","[37.866019739311, -122.266542125136]",4229.00
2,This art shop looked out of place next to the ...,https://drive.google.com/open?id=1ON0rVTsybNXE...,5699 Shattuck Ave.,This was the only apparent property that was f...,https://drive.google.com/open?id=13vvO_O3GiEu0...,608 Shattuck Ave.,The side of the block near Shattuck was more d...,NaN,NaN,NaN,NaN,NaN,NaN,"(37.8419909, -122.2648882)","(37.8419909, -122.2648882)","[35.130098959596, -119.460902121212]",NaN,NaN,4006.00
3,It seems that the wall was not designed for lo...,https://drive.google.com/open?id=1lq2-AA8zbS9U...,"2067-2071 University Ave, Berkeley, CA, 94704.",There were groups of homeless people outside t...,https://drive.google.com/open?id=1XZ7O9Iv-vJlp...,"2011 Berkeley Way, Berkeley, CA, 94704.","There are No-parking signs on the fence, but s...",https://drive.google.com/open?id=1xAhyRXCveyHC...,There is a large graffiti on the wall of the p...,"2016 Berkeley Way, Berkeley, CA, 94704.",https://drive.google.com/open?id=1IRztM7ZX4HzL...,"2005 Berkeley Way, Berkeley, CA, 94704.",The fence and iron gate shows that crimes may ...,"[37.8720195886659, -122.269816641442]","[37.8720195886659, -122.269816641442]","[37.8728409227947, -122.270248811613]","[37.8726479989409, -122.2704598844]","[37.8728723, -122.2704417]",4224.00
4,There were a few of these green dumpsters thro...,https://drive.google.com/open?id=1HNYxCgjW-eQQ...,"517 8th St, Oakland, CA, 94607","There were some ""pay and display"" parking tick...",https://drive.google.com/open?id=1kyhPjq_yANAl...,"530 8th St, Oakland, CA, 94607",This was one of four buildings that seemed aba...,https://drive.google.com/open?id=147w7g4FMLP4X...,I took a picture of this truck because it seem...,"810 Clay St, Oakland, CA 94607",https://drive.google.com/open?id=177FjcfjP9mOJ...,"512 8th St, Oakland, CA 94607",I was really surprised to find a temple on thi...,"[37.7964811, -122.2644011]","[37.7964811, -122.2644011]","[37.8009167156494, -122.275402395618]","[37.801272625156, -122.276025874901]","[37.8008494265101, -122.275235462283]",4031.00
5,View from the corner of Essex and Shattuck,https://drive.google.com/open?id=1CSHGT6G4Tayq...,"3045 Shattuck Ave, Berkeley, CA 94705",View facing Shattuck Ave,https://drive.google.com/open?i

In [7]:
to_drop = list(class_data.iloc[:1,np.arange(20,35)].columns)
class_data = class_data.drop(labels = to_drop, axis=1)

class_data = class_data.rename(columns= {
    'On a scale of 1 - 5, where 1 is "None" and 5 is "A Lot", how many empty beer or liquor bottles are visible in streets, yards, or alleys? ': 'Amount of Visible Empty Liquor Bottles [Scale: 1 (None) to 5 (A Lot)]', 
    'On a scale of 1 - 5, where 1 is "None" and 5 is "A Lot",  how many cigarette or cigar butts or discarded cigarette packages  are on the sidewalk or in the gutters? ': 'Amount of Visible Cigarette Debris [Scale: 1 (None) to 5 (A Lot)]',
    'On a scale of 1 - 5, where 1 is "None" and 5 is "A Lot",  how many condoms are present on the sidewalk, in the gutters, or street of block face? ': 'Amount of Visible Condoms [Scale: 1 (None) to 5 (A Lot)]',
    'On a scale of 1 - 5, where 1 is "None" and 5 is "A Lot", how much garbage, litter, or broken glass in the street or on the sidewalks? ': 'Amount of Visible Trash [Scale: 1 (None) to 5 (A Lot)]',
    'On a scale of 1-5 where 1 is "Friendly Responses / Greetings / Helpful" and 5 is "Treated with Suspicion", How were you regarded by the people in the block face?': 'Attitude of Residents [Scale: 1 (Friendly and Helpful) to 5 (Suspicious)]',
    'On a scale of 1 - 4, where 1 is "Very well kept / good condition" and 4 is "Poor / badly deteriorated condition", in general, how would you rate the condition of buildings on the block face? (includes residential buildings, recreational facilities, manufacturing plants, business / industrial headquarters, etc)' : 'Condition of Buildings [Scale: 1 (Well Kept) to 4 (Poor)]',
    'On a scale of 1 - 4, where 1 is "No fencing" and 4 is "High mesh fencing with barbed wire or spiked tops", is there fencing and what kind? (includes all property)': 'Presence of Fencing [Scale: 1 (No fencing) to 4 (High mesh with barbed wire or spiked tops)]',
    'On a scale of 1-3, where 1 is "Few or none" and 3 is a "Most/all of it", how many trees are linking the street of the block face? ': 'Amount of street linked with trees [Scale: 1 (None or Few) to 3 (Most or All)]',
})

class_data['What kinds of establishments are there on the block face? Select all that apply.'] = class_data['What kinds of establishments are there on the block face? Select all that apply.'].str.replace('Bodega, deli, corner-store, convenience store', 'Bodega deli corner-store convenience store')
class_data['What kinds of establishments are there on the block face? Select all that apply.'] = class_data['What kinds of establishments are there on the block face? Select all that apply.'].str.replace('Payday lenders, check cashers, or pawn shops', 'Payday lenders check cashers or pawn shops')
class_data['What kinds of establishments are there on the block face? Select all that apply.'] = class_data['What kinds of establishments are there on the block face? Select all that apply.'].str.replace('Professional offices \(doctor, dentist, lawyer, accountant, real estate\)', 'Professional offices (doctor dentist lawyer accountant real estate)')

class_data

,Unnamed: 0.1,Timestamp,Census Tract,Amount of Visible Empty Liquor Bottles [Scale: 1 (None) to 5 (A Lot)],Amount of Visible Cigarette Debris [Scale: 1 (None) to 5 (A Lot)],Amount of Visible Condoms [Scale: 1 (None) to 5 (A Lot)],Amount of Visible Trash [Scale: 1 (None) to 5 (A Lot)],Are there abandoned cars in the neighborhood? How many do you see?,Attitude of Residents [Scale: 1 (Friendly and Helpful) to 5 (Suspicious)],Condition of Buildings [Scale: 1 (Well Kept) to 4 (Poor)],...,What kinds of establishments are there on the block face? Select all that apply.,Amount of street linked with trees [Scale: 1 (None or Few) to 3 (Most or All)],Is there public transportation available in the block face?,Are private security guards visible?,Is there a police officer visible?,Image #1,"Full Address of Block Face in Image #1 (Street Number, Street Name, City, State, Zip Code). E.g.: 2128 Oxford Street, Berkeley, CA, 94704.",Image #3 coordinates,Image #4 coordinates,Image #5 coordinates
0,1,2018-10-21 22:08:57,4028.00,1.0,3.0,1.0,3.0,0.0,2.0,2.0,...,"Eating places/restaurants, Professional office...",2.0,0,0,0,https://drive.google.com/open?id=1F2vSKuf4hRQz...,"1633 San Pablo Ave, Oakland, CA","(37.8087183, -122.2715003)",NaN,NaN
1,2,2018-10-21 22:10:14,4229.00,2.0,2.0,1.0,2.0,0.0,4.0,2.0,...,"Eating places/restaurants, Professional office...",2.0,1,0,0,https://drive.google.com/open?id=1t7Cm2UkrM467...,"2150 Channing Way, Berkeley, CA, 94704","[37.8666288180719, -122.265876910725]","(37.8678899, -122.269247)","[37.866019739311, -122.266542125136]"
2,3,2018-10-21 22:23:08,4006.00,1.0,1.0,1.0,2.0,0.0,4.0,2.0,...,Liquor stores or Marijuana Dispensaries,2.0,1,0,1,https://drive.google.com/open?id=1K2U0ccdwjWB5...,5634 Shattuck Ave.,"[35.130098959596, -119.460902121212]",NaN,NaN
3,4,2018-10-22 00:01:58,4224.00,3.0,2.0,1.0,3.0,1.0,3.0,2.0,...,"Bars and alcoholic beverage services, Eating p...",2.0,1,0,0,https://drive.google.com/open?id=1bC5tf0RrhU8u...,"2067 University Ave, Berkeley, CA, 94704.","[37.8728409227947, -122.270248811613]","[37.8726479989409, -122.2704598844]","[37.8728723, -122.2704417]"
4,5,2018-10-22 00:08:49,4031.00,1.0,3.0,1.0,3.0,0.0,2.0,2.0,...,"Bodega deli corner-store convenience store, Ea...",1.0,0,0,0,https://drive.google.com/open?id=1DAEeW4g8askp...,"535 8th St, Oakland, CA, 94607","[37.8009167156494, -122.275402395618]","[37.801272625156, -122.276025874901]","[37.8008494265101, -122.275235462283]"
5,6,2018-10-22 00:09:49,4239.01,1.0,1.0,1.0,3.0,0.0,2.0,2.0,...,"Bars and alcoholic beverage services, Professi...",2.0,1,1,0,https://drive.google.com/open?id=1J7FBN9UWvZIh...,"3033 Shattuck Ave, Berkeley, CA 94705","[37.8536181290323, -122.266159419355]","[37.8541573333333, -122.266228]","[37.8537052419355, -122.26617166129]"
6,7,2018-10-22 10:52:35,4027.00,4.0,4.0,2.0,4.0,1.0,4.0,4.0,...,"Bars and alcoholic beverage services, Bodega d...",2.0,1,0,0,https://drive.google.com/open?id=1E4SxFgUUjtaR...,"2174 Brush St Oakland, California 94612","[37.8121013877551, -122.275144755102]","[37.8121013877551, -122.275144755102]","[37.8121013877551, -122.275144755102]"
7,8,2018-10-22 12:00:25,4014.00,1.0,2.0,1.0,4.0,1.0,3.0,3.0,...,"Community center, Playgrounds",1.0,1,0,1,https://drive.google.com/open?id=15opG7RDvlqB_...,"3236 Market St, Oakland, CA 94608","[37.8227030918367, -122.27581044898]","[37.8227030918367, -122.27581044898]","[37.8225462040816, -122.27585177551]"
8,9,2018-10-22 12:36:50,4239.01,3.0,1.0,1.0,3.0,0.0,2.0,2.0,...,"Gas station, Liquor stores or Marijuana Dispen...",2.0,1,0,0,https://drive.google.com/open?id=1yW4Qijl3MLsz...,"2001 Emerson St Berkeley, CA 94703","[37.854305, -122.269028]",NaN,NaN
9,10,2018-10-22 13:49:40,4239.01,1.0,2.0,1.0,2.0,0.0,2.0,2.0,...,NaN,3.0,0,0,0,https://drive.google.com/open?id=1GK9JXOqR5XUR...,NaN,"(37.8523001, -122.2675177)","[37.8522002682914, -122.266952668302]",NaN


In [8]:
class_data.shape

(139, 23)

In [9]:
image_data.to_csv("data/image_data_cleaned.csv", index=False)
class_data.to_csv("data/class_data_cleaned.csv", index=False)